In [20]:
# Step 1: Import packages
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

from qiskit import QuantumCircuit
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from qiskit.primitives import Estimator
from qiskit.algorithms.optimizers import ADAM

from qiskit_machine_learning.neural_networks import EstimatorQNN
from qiskit_machine_learning.algorithms import NeuralNetworkClassifier

In [31]:
import warnings
warnings.filterwarnings("ignore")

In [1]:
!pip install "qiskit==0.45.2" "qiskit-machine-learning==0.6.1" "qiskit-optimization==0.6.0" "qiskit-algorithms==0.2.1" "qiskit-aer"

In [22]:
# Step 2: Load and preprocess the Parkinson’s dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/parkinsons.data"
df = pd.read_csv(url)

X = df.drop(columns=['name', 'status'])  # Features
y = df['status']                         # Labels (0 or 1)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reduce dimensions to match qubit count (e.g., 4 qubits)
from sklearn.decomposition import PCA
pca = PCA(n_components=4)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [23]:
# Step 3: Define quantum circuit components
num_qubits = 4
feature_map = ZZFeatureMap(num_qubits)
ansatz = RealAmplitudes(num_qubits, reps=1)

In [25]:
# Step 4: Combine feature map and ansatz
composed_circuit = QuantumCircuit(num_qubits)
composed_circuit.compose(feature_map, inplace=True)
composed_circuit.compose(ansatz, inplace=True)

In [26]:
# Step 5: Build Estimator and QNN
estimator = Estimator()
qnn = EstimatorQNN(
    circuit=composed_circuit,
    input_params=feature_map.parameters,
    weight_params=ansatz.parameters,
    estimator=estimator
)

In [27]:
# Step 6: Define optimizer and classifier
optimizer = ADAM(maxiter=100, lr=0.01)
vqc = NeuralNetworkClassifier(neural_network=qnn, optimizer=optimizer, one_hot=False)

In [29]:
# Step 7: Train and evaluate
vqc.fit(X_train, y_train.to_numpy())
y_pred = vqc.predict(X_test)

In [32]:
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         0
         0.0       0.00      0.00      0.00         7
         1.0       0.84      0.50      0.63        32

    accuracy                           0.41        39
   macro avg       0.28      0.17      0.21        39
weighted avg       0.69      0.41      0.51        39

